# Debugging and Testing



## Background

In this project, we continue to work towards a replication of Cunha, Heckman and Schennach (CHS), Econometrica, 2010. This time we implement Kalman filters that can be used to make maximum likelihood estimation of Dynamic Latent factor models computationally feasible. Moreover, we learnt how to debug more efficiently and how to use Pytest to write unit tests for my functions. 

Kalman filters can be used for state and parameter estimation in so called state space models. They combine several measurements (in our case observed test scores) of an underlying state vector (in our case latent skills) in an optimal way. This even works if the state vector is not static (skills change from one period to the next). 

To make the relationship to the technology of skill formation and general state space models clearer, we used the following notation:

Collect of latent factors in period $t$ in the vetcor $\mathbf{x}_t$:

$\mathbf{x}_t = (cognitive\_skills_t, \text{ } noncognitive\_skills_t, \text{ } investments_t, \text{ } parents\_cognitive\_skills_t, parents\_noncognitive\_skills_t)$

Collect all measurements in period $t$ in the vector $\mathbf{y}_t$

$\mathbf{y}_t = (IQ\_score_t, \text{ } math\_score_t, bpi\_antisocial_t, bpi\_anxiety_t, \ldots)$

Then the full model can be expressed by two fundamental equations:

$$\begin{align}
        \mathbf{x}_{t + 1} &= \mathbf{F}_t(\mathbf{x}_t) + \boldsymbol{\eta}_{t} &\quad{\text{transition equations}} \\
        \mathbf{y}_t &= \mathbf{H}_t \cdot \mathbf{x}_t + \boldsymbol{\epsilon}_t &\quad{\text{linear measurement equations}}
\end{align}$$

where $\mathbf{F}_t(\mathbf{x}_t)$ are the production functions of skills, $\mathbf{H}_t$ are the factor loadings that describe the relationship between observed measurements and latent skills, $\boldsymbol{\eta}_{t}$ is a vector of skill shocks and $\boldsymbol{\epsilon}_t$ is a vector of measurement errors.

You can find a detailed introduction to Kalman filters in `kalman_filters.pdf` . 

Our main tasks of this project are to debug the predict step of the Kalman filter and to test and implement the update step. 

The *update step* takes an initial estimate of the state vector (skills) and refines it using a *measurement* (test scores).  It has the following interface.


``` python

def square_root_linear_update(state, root_cov, measurement, loadings):
    """Update *state* and *root_cov with* with a *measurement*.
    
    Args:
        state (pd.Series): pre-update estimate of the unobserved state vector
        root_cov (pd.DataFrame): lower triangular matrix square-root of the
            covariance matrix of the state vector before the update
        measurement (float): the measurement to incorporate
        loadings (pd.Series): the factor loadings
        
    Returns:
        updated_state (pd.Series)
        updated_root_cov (pd.Series)
    
    """

    return updated_state, updated_root_cov

```

The *predict step* takes the refined estimate of the state vector and uses the law of motion of the state vector (production functions of skills) to produce an estimate for the state vector in the next period. It has the following interface:

``` python 

def square_root_unscented_predict(state, root_cov, params, shock_sds, kappa):
    """Predict *state* in next period and adjust *root_cov*.

    Args:
        state (pd.Series): period t estimate of the unobserved state vector
        root_cov (pd.DataFrame): lower triangular matrix square-root of the
            covariance matrix of the state vector in period t
        params (dict): keys are the names of the states (latent
            factors), values are series with parameters for the transition
            equation of that state.
        shock_sds (pd.Series): standard deviations of the shocks
        kappa (float): scaling parameter for the unscented predict

    Returns:
        predicted_state (pd.Series)
        predicted_root_cov (pd.DataFrame)

    """

    return predicted_state, predicted_root_cov

```






## Steps we performed



1. We opened a command line in the `code` folder and ran all tests. At the beginning, we wrote two tests in `test_predict.py`. Both fail because the functions in `predict.py` contain bugs. 

2. We then used a debugger to debug the "nice" bugs. By nice bugs we mean the ones that lead to exceptions and therefore cannot be overlooked. 

3. We then wrote more unit tests to locate the "dangerous" bugs. By dangerous we mean the ones where the code runs through but produces wrong results. 


4. We wrote tests for `square_root_linear_update` in the module `test_update.py`. 

    **Notes**
    - To generate test cases, it is easier for us to use the formulae for the normal (i.e. non square-root) linear update. The resulting covariance matrix from the `square_root_linear_update` can easily be transformed to the full covariance matrix for testing.
    - We do not have too many tests, we think it is often better to have a few well designed tests, than many tests that all cover the same cases. 
 
5. We build the code `update.py` based on the model and implement `square_root_linear_update` in `update.py`. We sticked to the exact interface from above. 

6. All tests pass.


